In [64]:
import os
import re 
import glob
import time
import numpy as np
import pandas as pd
from collections import Counter

In [101]:
os.getcwd()

'/home/kaline/doccano-fork-and-tools/ncdj-tools/export_utility'

In [99]:
! cd NER_PRECEDENTE_EXPORT/
! ls

'01 - Processamento.ipynb'	    Majority.ipynb
'02 - CRF.ipynb'		    mock
'03 - Precedentes.ipynb'	    NER
 combined_csv.csv		    ner_dataset.csv
 consolidate_all_ner_documents.py   ner_dataset.csv.zip
 crf_tagger.txt			    NER_PRECEDENTE_EXPORT
 df_prec.csv			    NER_PRECEDENTE_EXPORT.tar.bz2
 DUMP.ipynb			    preprocessados.csv
 evrcombined_csv.csv		    resultado.csv
'Explorando NER.ipynb'		    TesteVisu.ipynb
 exporter_ner.py		    xtrain.txt
 exporter.py			    ytrain.txt


In [81]:
# /home/kaline/doccano-fork-and-tools/tmp/NER_PRECEDENTE_EXPORT
# /NER_PRECEDENTE_EXPORT/161704902/[PRATICA_ETAPA_1]/Precedente
extension = 'csv'
all_filenames = [i for i in glob.glob('/NER_PRECEDENTE_EXPORT/*/**/***/****/*****.{}'.format(extension))]

In [82]:
len(all_filenames)
all_filenames[:3]

[]

In [92]:
file = 'doccano-fork-and-tools/ncdj-tools/export_utility/NER_PRECEDENTE_EXPORT/161704902/[PRATICA_ETAPA_1]/Precedente/2c63e06285e4.ner.csv'

In [93]:
pd.read_csv(file,delimiter=';', na_values='NaN') 

FileNotFoundError: [Errno 2] File b'doccano-fork-and-tools/ncdj-tools/export_utility/NER_PRECEDENTE_EXPORT/161704902/[PRATICA_ETAPA_1]/Precedente/2c63e06285e4.ner.csv' does not exist: b'doccano-fork-and-tools/ncdj-tools/export_utility/NER_PRECEDENTE_EXPORT/161704902/[PRATICA_ETAPA_1]/Precedente/2c63e06285e4.ner.csv'

In [68]:
# Cria uma tag de inicio e fim de arquivo em cada 'csv' antes de apendar todos eles.

frames = []
for all_files in all_filenames:
    df = pd.read_csv(all_files,delimiter=';', na_values='NaN') # Lê o arquivo
    frames.append(df) # Adiciona esse dataframe no 'dataframe maior'
    
df_prec = pd.concat(frames).reset_index(drop=True)

In [69]:
df_prec.to_csv("df_prec.csv",index=False,encoding='utf-8') # Cria um arquivo com todas as anotações.
df_prec['Token'] = df_prec['Token'].astype('str')

In [70]:
df_prec.head(30)

,Token,Tag
0,RTJ,B_Classe - Outros
1,,I_Classe - Outros
2,87/758,B_Número
3,HC,B_Classe - Writ
4,,I_Classe - Writ
5,119.115,B_Número
6,/,I_Número
7,MG,B_Origem
8,",",I_Origem
9,,O


In [56]:
df_prec.Tag.unique()

array(['B_Classe - Outros', 'I_Classe - Outros', 'B_Número',
       'B_Classe - Writ', 'I_Classe - Writ', 'I_Número', 'B_Origem',
       'I_Origem', 'O', 'B_Relator', 'I_Relator', 'B_Data de Julgamento',
       'B_Classe - Recursal', 'I_Classe - Recursal', 'B_Tipo', 'I_Tipo',
       'B_Classe - Constitucional', 'I_Classe - Constitucional',
       'I_Data de Julgamento', 'B_Data de Publicação',
       'I_Data de Publicação', 'B_Classe - Súmula', 'I_Classe - Súmula',
       'B_Corte', 'I_Corte', 'B_Classe - Penal originária',
       'I_Classe - Penal originária', 'B_Pessoa - Não Relator',
       'I_Pessoa - Não Relator'], dtype=object)

In [57]:
df_prec[df_prec.Tag.str.endswith('_Número')].head(20)

,Token,Tag
2,87/758,B_Número
5,119.115,B_Número
6,/,I_Número
29,84.448/2011,B_Número
32,655.217,B_Número
33,/,I_Número
50,489108,B_Número
51,",",I_Número
78,1.802,B_Número
79,",",I_Número


# Ideia para separação de sentenças:

-- Obter uma tag inicial que começa com B_, a sentença dura até que 'I_(fim da tag inicial)' se altere

In [58]:
df_prec.head(5)

,Token,Tag
0,RTJ,B_Classe - Outros
1,,I_Classe - Outros
2,87/758,B_Número
3,HC,B_Classe - Writ
4,,I_Classe - Writ


In [63]:
# Conferindo se nao pode existir o caso onde uma Tag que começa com 'I' é seguida de outra Tag que começa
# com 'I'
lista = []
for i in range(len(df_prec)):
    if df_prec.Tag.iloc[i].startswith('I') == True and df_prec.Tag.iloc[i] != df_prec.Tag.iloc[i+1] == True:
        lista = df_prec.Tag.iloc[i+1]
#         print(df_prec.Tag.iloc[i+1])
lista

[]